# KAIST AI605 Assignment 2: Text Retrieval

## Environment
You will only use Python 3.7 and PyTorch 1.9, which is already available on Colab:

In [1]:
import re
import os
import torch

from textwrap import wrap, indent
from collections import Counter, defaultdict

from platform import python_version
from IPython.display import clear_output

print(f"python {python_version()}")
print(f"torch {torch.__version__}")

python 3.7.12
torch 1.9.0+cu111


You will use two datasets, namely SQuAD, for retrieval.
Note that this is a MRC dataset but we will view them as retrieval task by trying to find the correct document corresponding to the question among all the documents in the **validation** data. 

In [2]:
try:
    import datasets
except ImportError:
    !pip install datasets

In [3]:
from datasets import load_dataset

squad = load_dataset("squad")
valid0 = squad["validation"][0]
clear_output()
print(f"Context:")
print(indent("\n".join(wrap(valid0["context"])), "  "))
print(f"Question:")
print(indent("\n".join(wrap(valid0["question"])), "  "))

Context:
  Super Bowl 50 was an American football game to determine the champion
  of the National Football League (NFL) for the 2015 season. The
  American Football Conference (AFC) champion Denver Broncos defeated
  the National Football Conference (NFC) champion Carolina Panthers
  24–10 to earn their third Super Bowl title. The game was played on
  February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at
  Santa Clara, California. As this was the 50th Super Bowl, the league
  emphasized the "golden anniversary" with various gold-themed
  initiatives, as well as temporarily suspending the tradition of naming
  each Super Bowl game with Roman numerals (under which the game would
  have been known as "Super Bowl L"), so that the logo could prominently
  feature the Arabic numerals 50.
Question:
  Which NFL team represented the AFC at Super Bowl 50?


In [4]:
def preprocess(squad):
    context_dict = {}
    documents = []
    query_pairs = []
    count = 0

    for context, question in zip(squad["context"], squad["question"]):
        context = context.strip()
        question = question.strip()
        if context in context_dict:
            context_id = context_dict[context]
        else:
            context_id = count
            context_dict[context] = count
            count += 1
            documents.append(context)
        query_pairs.append((context_id, question))
    return documents, query_pairs

squad_documents, squad_queries = preprocess(squad["validation"])

## 1. Measuring Similarity
We discussed in Lecture 08 that there are several ways to measure similarity between two vectors,
such as L2 (Euclidean) distance, L1 (Manhattan) distance, inner product, and cosine distance.
Here, except for inner product, all other measures are [*metric*](https://en.wikipedia.org/wiki/Metric_(mathematics)).

> A function $d: X \times X \to \mathbb{R}$ is said to be a metric on $X$ if:
> - (*Non-negativity*) $d(\mathbf{x}, \mathbf{y}) \geq 0$ for all $\mathbf{x}, \mathbf{y} \in X$
> 
> And satisfies following three axioms:
> 1. (*Identity of Indiscernibility*) $d(\mathbf{x}, \mathbf{y}) = 0 \iff \mathbf{x} = \mathbf{y}$
> 2. (*Symmetry*) $d(\mathbf{x}, \mathbf{y}) = d(\mathbf{y}, \mathbf{x})$ for all $\mathbf{x}, \mathbf{y} \in X$
> 3. (*Triangle Inequality*) $d(\mathbf{x}, \mathbf{y}) \leq d(\mathbf{x}, \mathbf{z}) + d(\mathbf{z}, \mathbf{y})$ for all $\mathbf{x}, \mathbf{y}, \mathbf{z} \in X$

> **Problem 1.1** *(2 points)* Using the definition of metric above, prove that L1 distance is a metric.

> **Solution 1.1**
> Let $\mathbf{x}, \mathbf{y}, \mathbf{z} \in \mathbb{R}^n$ and $\mathbf{x} = (x_1, \dots, x_n), \mathbf{y} = (y_1, \dots, y_n), \mathbf{z} = (z_1, \dots, z_n)$.
> 
> L1 distance is defined as
> $$
    d(\mathbf{x}, \mathbf{y})
        := |\mathbf{x} - \mathbf{y}|
         = \sum_{i=1}^n |x_i - y_i|. $$
> To prove that L1 distance is a metric on $\mathbb{R}^n$,
> we should show L1 distance is non-negative and satisfies above three properties.
> 
> (*Non-negativity*)
> By the non-negative property of the absolute value,
> $$
    |x_i - y_i| \geq 0 \, \text{ for } \, i = 1, \dots, n. $$
> Then,
> $$
    \begin{align*}
        d(\mathbf{x}, \mathbf{y})
        &= \sum_{i=1}^n |x_i - y_i| \\
        &\geq \sum_{i=1}^n 0 \\
        &= 0.
    \end{align*} $$
> So, $d(\mathbf{x}, \mathbf{y}) \geq 0$ for all $\mathbf{x}, \mathbf{y} \in \mathbf{R}^n$ and L1 distance is non-negative.
> 
> (*Identity of Indiscernibility*)
> By the non-negative property of the absolute value,
> $$
    d(\mathbf{x}, \mathbf{y}) = \sum_{i=1}^n |x_i - y_i| = 0
    \iff |x_i - y_i| = 0 \,\text{ for }\, i = 1, \dots, n. $$
> And, by the identity of indiscernibles property of the absolute value,
> $$
    \begin{align*}
        |x_i - y_i| = 0 \,\text{ for }\, i = 1, \dots, n
        &\iff x_i = y_i \,\text{ for }\, i = 1, \dots, n \\
        &\iff \mathbf{x} = \mathbf{y}.
    \end{align*} $$
> So, $d(\mathbf{x}, \mathbf{y}) = 0 \iff \mathbf{x} = \mathbf{y}$ and L1 distance satisfies the identity of indiscernibles axiom.
> 
> (*Symmetry*)
> By the definition of the absolute value,
> $$
    \begin{align*}
        d(\mathbf{x}, \mathbf{y})
        &= \sum_{i=1}^n |x_i - y_i| \\
        &= \sum_{i=1}^n |-(x_i - y_i)| \\
        &= \sum_{i=1}^n |y_i - x_i| \\
        &= d(\mathbf{y}, \mathbf{x}).
    \end{align*} $$
> So, $d(\mathbf{x}, \mathbf{y}) = d(\mathbf{y}, \mathbf{x})$ for all $\mathbf{x}, \mathbf{y} \in \mathbf{R}^n$ and L1 distance satisfies the symmetry axiom.
>
> (*Triangle Inequality*)
> By the triangle inequality property of the absolute value,
> $$
    |(x_i - z_i) + (z_i - y_i)| \leq |x_i - z_i| + |z_i - y_i|\, \text{ for }\, i = 1, \dots, n. $$
> Then,
> $$
    \begin{align*}
        d(\mathbf{x}, \mathbf{y})
        &= \sum_{i=1}^n |x_i - y_i| \\
        &= \sum_{i=1}^n |(x_i - z_i) + (z_i - y_i)| \\
        &\leq \sum_{i=1}^n (|x_i - z_i| + |z_i - y_i|) \\
        &\leq \sum_{i=1}^n |x_i - z_i| + \sum_{i=1}^n |z_i - y_i| \\
        &= d(\mathbf{x}, \mathbf{z}) + d(\mathbf{z}, \mathbf{y}).
    \end{align*} $$
> So, $d(\mathbf{x}, \mathbf{y}) \leq d(\mathbf{x}, \mathbf{z}) + d(\mathbf{z}, \mathbf{y})$ for all $\mathbf{x}, \mathbf{y}, \mathbf{z} \in \mathbf{R}^n$ and L1 distance satisfies the triangle inequality axiom.
>
> Finally, we show that L1 distance is non-negative and satisfies three axioms of a metric.
> Thus, L1 distance is a metric on $\mathbf{R}^n$.

> **Problem 1.2** *(2 points)* Prove that negative inner product is NOT a metric.

> **Solution 1.2**
> Let $\mathbf{x}, \mathbf{y}, \mathbf{z} \in \mathbb{R}^n$ and $\mathbf{x} = (x_1, \dots, x_n), \mathbf{y} = (y_1, \dots, y_n), \mathbf{z} = (z_1, \dots, z_n)$.
> 
> Negative inner product is defined as
> $$
    d(\mathbf{x}, \mathbf{y})
        := - \langle\mathbf{x}, \mathbf{y}\rangle
         = - \sum_{i=1}^n x_i y_i. $$
> 
> To prove that negative inner product is not a metric on $\mathbb{R}^n$,
> we should show negative inner product is not non-negative or not satisfies one of above three properties.
>
> Let assume $\mathbf{x} = (1, 1), \mathbf{y} = (0, 0)$. Then,
> $$
    \begin{align*}
        d(\mathbf{x}, \mathbf{y})
             &= - \sum_{i=1}^n x_i y_i \\
             &= - (1 \cdot 0 + 1 \cdot 0) \\
             &= 0.
    \end{align*} $$
> It means there exists $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ that NOT $d(\mathbf{x}, \mathbf{y}) = 0 \iff \mathbf{x} = \mathbf{y}$.
> Thus, negative inner product is not a metric because it does not satisfy the identity of indiscernibles axiom.

> **Problem 1.3** *(2 points)* Prove that cosine distance (1 - cosine similarity) is NOT a metric.

> **Solution 1.3**
> Let $\mathbf{x}, \mathbf{y}, \mathbf{z} \in \mathbb{R}^n$ and $\mathbf{x} = (x_1, \dots, x_n), \mathbf{y} = (y_1, \dots, y_n), \mathbf{z} = (z_1, \dots, z_n)$.
> 
> Cosine distance is defined as
> $$
    d(\mathbf{x}, \mathbf{y})
        := 1 - \cos(\theta)
         = 1 - \frac{\mathbf{x} \cdot \mathbf{y}}{\Vert\mathbf{x}\Vert \Vert\mathbf{y}\Vert}
         = 1 - \frac{\sum_{i=1}^n x_i y_i}{\sqrt{\sum_{i=1}^n x_i^2} \sqrt{\sum_{i=1}^n y_i^2}}. $$
> 
> To prove that cosine distance is not a metric on $\mathbb{R}^n$,
> we should show cosine distance is not non-negative or not satisfies one of above three properties.
>
> Let assume $\mathbf{x} = (1, 0), \mathbf{y} = (0, 1), \mathbf{z} = (1, 1)$.
> Then,
> $$
    \begin{align*}
        d(\mathbf{x}, \mathbf{y})
        &= 1 - \frac{\sum_{i=1}^n x_i y_i}{\sqrt{\sum_{i=1}^n x_i^2} \sqrt{\sum_{i=1}^n y_i^2}} \\
        &= 1 - \frac{1 \cdot 0 + 0 \cdot 1}{\sqrt{1^2 + 0^2} \sqrt{0^2 + 1^2}} \\
        &= 1.
    \end{align*} $$
> And,
> $$
    \begin{align*}
        d(\mathbf{x}, \mathbf{z}) + d(\mathbf{z}, \mathbf{y})
        &= \left(1 - \frac{\sum_{i=1}^n x_i z_i}{\sqrt{\sum_{i=1}^n x_i^2} \sqrt{\sum_{i=1}^n z_i^2}} \right) + \left(1 - \frac{\sum_{i=1}^n z_i y_i}{\sqrt{\sum_{i=1}^n z_i^2} \sqrt{\sum_{i=1}^n y_i^2}} \right) \\
        &= \left(1 - \frac{1 \cdot 1 + 0 \cdot 1}{\sqrt{1^2 + 0^2} \sqrt{1^2 + 1^2}}\right) + \left(1 - \frac{0 \cdot 1 + 1 \cdot 1}{\sqrt{1^2 + 1^2} \sqrt{0^2 + 1^2}}\right)\\
        &= 2 - \sqrt{2} \\
        &< 1.
    \end{align*} $$
> It means there exists $\mathbf{x}, \mathbf{y}, \mathbf{z} \in \mathbf{R}^n$ that $d(\mathbf{x}, \mathbf{y}) > d(\mathbf{x}, \mathbf{z}) + d(\mathbf{z}, \mathbf{y})$.
> Thus, cosine distance is not a metric because it does not safisfy the triangular inequality axiom.

> **Problem 1.4 (bonus)** *(3 points)*
Given a model that can perform nearest neighbor search in L2 space,
can you modify your query and your key vectors to perform maximum inner product search?

> **Solution 1.4**
> Let query be $\mathbf{q} = (q_1, \dots, q_n) \in \mathbb{R}^n$,
> key be $\mathbf{k} = (k_1, \dots, k_n) \in \mathbb{R}^n$,
> and the set of candidate keys be $K = \{\mathbf{k} | \mathbf{k} \in \mathbb{R}^n\} \subseteq \mathbb{R}^n$.
>
> L2 NNS is defined as
> $$
    \begin{align*}
        \text{NNS}(\textbf{q}, K)
            &:= \argmin_{\textbf{k} \in K} \Vert \textbf{q} - \textbf{k} \Vert_2^2 \\
            &= \argmin_{\textbf{k} \in K} \sqrt{\sum_{i=1}^n (q_i - k_i)^2} \\
            &= \argmin_{\textbf{k} \in K} \sum_{i=1}^n (q_i - k_i)^2 \\
            &= \argmin_{\textbf{k} \in K} \sum_{i=1}^n (- q_i k_i + \frac{1}{2}k_i^2) \\
            &= \argmax_{\textbf{k} \in K} (\sum_{i=1}^n q_i k_i - \frac{1}{2} \Vert \textbf{k} \Vert_2^2).
    \end{align*} $$
> And MIPS is defined as
> $$
    \begin{align*}
        \text{MIPS}(\textbf{q}, K)
            &:= \argmax_{\textbf{k} \in K} \langle \textbf{q}, \textbf{k} \rangle \\
            &= \argmax_{\textbf{k} \in K} \sum_{i=1}^n q_i k_i.
    \end{align*} $$
> Notice that L2 NNS becomes MIPS when $\Vert \textbf{k} \Vert_2^2$ is constant for all $\textbf{k} \in K$.
> And it means the norm of key vectors should be same.
>
> Now, let the largest norm of key vectors be $d = \max_{\textbf{k} \in K} \Vert\textbf{k}\Vert_2$.
> And let the new key be $\tilde{\textbf{k}} = (\frac{\textbf{k}}{d}, \frac{1}{2} - \Vert\frac{\textbf{k}}{d}\Vert_2^2, \frac{1}{2} - \Vert\frac{\textbf{k}}{d}\Vert_2^4, \dots, \frac{1}{2} - \Vert\frac{\textbf{k}}{d}\Vert_2^{2^m})$,
> and the new query be $\tilde{\textbf{q}} = (\textbf{q}, 0, 0, \dots, 0)$ which has same dimension with $\tilde{\textbf{k}}$.
> Then L2 NNS of the new query and new keys becomes:
> $$
    \begin{align*}
        \text{NNS}(\tilde{\textbf{q}}, \tilde{K})
            &= \argmax_{\textbf{k} \in K} (\sum_{i=1}^{n + m} \tilde{q}_i \tilde{k}_i - \frac{1}{2} \Vert\tilde{\textbf{k}}\Vert_2^2) \\
            &= \argmax_{\textbf{k} \in K} (\sum_{i=1}^n q_i k_i - \frac{d}{2} \Vert\tilde{\textbf{k}}\Vert_2^2).
    \end{align*} $$
> And,
> $$
    \begin{align*}
        \Vert\tilde{\textbf{k}}\Vert_2^2
            &= \Vert\frac{\textbf{k}}{d}\Vert_2^2 + (\frac{1}{2} - \Vert\frac{\textbf{k}}{d}\Vert_2^2)^2 + (\frac{1}{2} - \Vert\frac{\textbf{k}}{d}\Vert_2^4)^2 + \cdots + (\frac{1}{2} - \Vert\frac{\textbf{k}}{d}\Vert_2^{2^m})^2 \\
            &= \Vert\frac{\textbf{k}}{d}\Vert_2^2 + \frac{m}{4} - (\Vert\frac{\textbf{k}}{d}\Vert_2^2 + \Vert\frac{\textbf{k}}{d}\Vert_2^4 + \cdots + \Vert\frac{\textbf{k}}{d}\Vert_2^{2^m}) + (\Vert\frac{\textbf{k}}{d}\Vert_2^4 + \Vert\frac{\textbf{k}}{d}\Vert_2^8 + \cdots + \Vert\frac{\textbf{k}}{d}\Vert_2^{2^{m+1}}) \\
            &= \frac{m}{4} + \Vert\frac{\textbf{k}}{d}\Vert_2^{2^{m+1}}.
    \end{align*} $$
> When $m$ goes to infinity, the norm of the new key becomes:
> $$
    \Vert\tilde{\textbf{k}}\Vert_2^2 = \frac{m}{4} + \Vert\frac{\textbf{k}}{d}\Vert_2^{2^{m+1}} \approx \frac{m}{4}, $$
> since $0 \leq \Vert\frac{\textbf{k}}{d}\Vert_2 \leq 1$.
> So, the norm of the new key is constant for all $\textbf{k} \in K$. 
> Again, L2 NNS becomes:
> $$
    \begin{align*}
        \text{NNS}(\tilde{\textbf{q}}, \tilde{K})
            &= \argmax_{\textbf{k} \in K} (\sum_{i=1}^n q_i k_i - \frac{d}{2} \Vert\tilde{\textbf{k}}\Vert_2^2) \\
            &\approx \argmax_{\textbf{k} \in K} (\sum_{i=1}^n q_i k_i - \frac{dm}{8}) \\
            &= \argmax_{\textbf{k} \in K} \sum_{i=1}^n q_i k_i \\
            &= \text{MIPS}(\textbf{q}, K).
    \end{align*} $$
> So, we can perform MIPS with a model performs L2 NNS.
>
> Reference: [Asymmetric LSH(ALSH) for Sublinear Time Maximum Inner Product Search (MIPS)](https://arxiv.org/pdf/1405.5869.pdf)

We first create an abstract class for performing similarity search as follows:

In [5]:
class SimilaritySearch:
    def __init__(self):
        self.is_trained = False

    def _check_trained(self):
        if not self.is_trained:
            raise RuntimeError("Model should be trained before adding documents")

    # Make vocab from documents
    def train(self, documents: list):
        raise NotImplementedError()

    # Add documents (a list of text)
    def add(self, documents: list):
        raise NotImplementedError()

    # Returns the indices of top-k documents among the added documents
    # that are most similar to the input query 
    def search(self, query: str, k: int) -> list:
        raise NotImplementedError()

You will use the same space-based tokenizer that you used in Assignment 1, with lowercasing to make it case insensitive.

In [6]:
def tokenize(sentence):
    tokens = [word for word in re.split(r"\W+", sentence.lower()) if word]
    return tokens

## 2. Sparse Search



In [7]:
def recall(model: SimilaritySearch, queries: list, k: int):
    hit, total = 0, 0
    for idx, query in queries:
        topk_idx = model.search(query, k)
        hit += (1 if idx in topk_idx else 0)
        total += 1
    return hit / total

> **Problem 2.1** *(2 points)*
We will first start with Bag of Words that we discussed in Lecture 08.
Using the definition in the class, implement `BagOfWords` class that subclasses `SimilaritySearch` class.

In [8]:
class BagOfWords(SimilaritySearch):
    def __init__(self):
        super().__init__()
        self.vocab = defaultdict(int)
        self.vocab2idx = defaultdict(lambda: -1)
        self.dtm = None

    def _normalize(self, vector: torch.tensor) -> torch.tensor:
        norm_vector = vector / torch.norm(vector)
        return norm_vector

    def _embed(self, document: str) -> torch.tensor:
        counts = Counter(tokenize(document))
        idxs = [self.vocab2idx[token] for token in counts]
        cols = torch.tensor([[idx for idx in idxs if idx >= 0]], requires_grad=False)
        values = torch.tensor([count for idx, count in zip(idxs, counts.values())
                               if idx >= 0], dtype=float, requires_grad=False)
        size = [len(self.vocab)]
        embed_vector = torch.sparse_coo_tensor(cols, self._normalize(values),
                                               size, requires_grad=False)
        return embed_vector

    def train(self, documents: list):
        self.dtm = None
        for document in documents:
            for token in set(tokenize(document)):
                self.vocab[token] += 1
        self.vocab2idx.update({token: idx for idx, token in enumerate(self.vocab)})
        self.is_trained = True

    def add(self, documents: list):
        self._check_trained()
        sub_dtm = torch.stack([self._embed(document) for document in documents])
        self.dtm = sub_dtm if self.dtm is None else torch.cat((self.dtm, sub_dtm))
        self.dtm = self.dtm.to_dense()  # For speed up

    def search(self, query: str, k: int) -> list:
        query_vector = self._embed(query).coalesce()
        query_idxs, query_values = query_vector.indices(), query_vector.values()
        dtm = self.dtm.index_select(dim=1, index=query_idxs[0])
        scores = torch.matmul(dtm, query_values)
        scores = (scores.to_dense() if scores.is_sparse else scores)
        topk_idx = scores.topk(k).indices.tolist()
        return topk_idx


In [9]:
bow = BagOfWords()
bow.train(squad_documents)
bow.add(squad_documents)
topk_idx = bow.search(squad_documents[0], 10)
print(topk_idx)

[0, 24, 1, 28, 36, 398, 1000, 1373, 12, 850]


In [10]:
print(f"Recall@10 of BagOfWords: {recall(bow, squad_queries, 10):.4f}")

Recall@10 of BagOfWords: 0.5562


> **Problem 2.2** *(2 points)* Using the definition in Lecture 08, implement `TFIDF` class that subclasses `BagOfWords` class.

In [11]:
class TFIDF(BagOfWords):
    def __init__(self):
        super().__init__()
        self.idf = None

    def _normalize(self, vector: torch.tensor) -> torch.tensor:
        norm_vector = vector / torch.sum(vector)
        return norm_vector

    def _scores(self, tf: torch.tensor, idf: torch.tensor) -> torch.tensor:
        return torch.sum((tf * idf), dim=1)

    def train(self, documents: list):
        super().train(documents)
        df = torch.tensor(list(self.vocab.values()), dtype=float)
        self.idf = torch.log(len(documents) / df).unsqueeze(dim=0)

    def search(self, query: str, k: int) -> list:
        words = set(tokenize(query))
        query_idxs = torch.tensor([idx for idx, token in enumerate(self.vocab)
                                   if token in words])
        tf = self.dtm.index_select(dim=1, index=query_idxs)
        tf = (tf.to_dense() if tf.is_sparse else tf)
        idf = self.idf.index_select(dim=1, index=query_idxs)
        scores = self._scores(tf, idf)
        topk_idx = scores.topk(k).indices.tolist()
        return topk_idx

In [12]:
tfidf = TFIDF()
tfidf.train(squad_documents)
tfidf.add(squad_documents)
topk_idx = tfidf.search(squad_documents[0], 10)
print(topk_idx)

[0, 5, 7, 24, 4, 23, 25, 8, 22, 26]


> **Problem 2.3** *(2 points)* Use `TFIDF` to measure the recall rate of the correct document when 10 documents (contexts) are retrieved (this is called **Recall@10**) in SQuAD **validation** set.

In [13]:
print(f"Recall@10 of TFIDF: {recall(tfidf, squad_queries, 10):.4f}")

Recall@10 of TFIDF: 0.8605


> **Problem 2.4 (bonus)** *(2 points)* Implement `BM25` that sublcasses `BagOfWords` and repeat Problem 2.3 for BM25.

In [14]:
class BM25(TFIDF):
    def __init__(self, k1: float = 1.2, b: float = 0.75):
        super().__init__()
        self.dl = None
        self.k1 = k1
        self.b = b
    
    def _scores(self, tf: torch.tensor, idf: torch.tensor) -> torch.tensor:
        return torch.sum(idf * ((tf * (self.k1 + 1) / (tf + self.k1 * self.dlw))), dim=1)

    def train(self, documents: list):
        super().train(documents)
        dl = torch.tensor([len(tokenize(document)) for document in documents], dtype=float)
        self.dlw = (1 - self.b + self.b * dl / torch.mean(dl)).unsqueeze(dim=1)

In [15]:
bm25 = BM25()
bm25.train(squad_documents)
bm25.add(squad_documents)
topk_idx = bm25.search(squad_documents[0], 10)
print(topk_idx)

[0, 5, 23, 4, 7, 20, 25, 472, 24, 2]


In [16]:
print(f"Recall@10 of BM25: {recall(bm25, squad_queries, 10):.4f}")

Recall@10 of BM25: 0.7365


## 3. Dense Search

To obtain the embedding of each document and query, you will use GloVe embedding (Pennington et al., 2014).
Go to [GloVe project page](https://nlp.stanford.edu/projects/glove/) and download `glove.6B.zip`.
You are also welcome to use other more convenient ways to access the GloVe embeddings.
You will compute the document's embedding by simply averaging the embeddings of all words in the document (same for the query), and then normalizing it.
This way, inner product effectively becomes cosine similarity.

In [17]:
from urllib import request
import zipfile

if not os.path.exists("glove/glove.6B.100d.txt"):
    os.makedirs("glove", exist_ok=True)
    print("Downloading glove.6B.zip ... ")
    request.urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", "glove/glove.6B.zip")
    with zipfile.ZipFile("glove/glove.6B.zip", "r") as zipf:
        zipf.extractall("glove")
    clear_output()

dim = 300

with open(f"glove/glove.6B.{dim}d.txt", "r") as glove:
    glove_map = map(lambda line: line.split(), glove.readlines())
    to_tensor = lambda float_list: torch.tensor(list(map(float, float_list)))
    word2vec = {word: to_tensor(vec) for word, *vec in glove_map}

In [18]:
class GloVeAverage(SimilaritySearch):
    def __init__(self, word2vec: dict):
        super().__init__()
        self.word2vec = word2vec
        self.dtm = None

    def _embed(self, document: str) -> torch.tensor:
        word_vecs = torch.vstack([self.word2vec[word] for word in tokenize(document)
                                  if word in word2vec])
        vector = torch.mean(word_vecs, dim=0)
        embed_vector = vector / torch.norm(vector)
        return embed_vector

    def train(self, documents: list):
        self.dtm = None
        self.is_trained = True  # No need to make vocab

    def add(self, documents: list):
        self._check_trained()
        sub_dtm = torch.stack([self._embed(document) for document in documents])
        self.dtm = sub_dtm if self.dtm is None else torch.cat((self.dtm, sub_dtm))

    def search(self, query: str, k: int) -> list:
        query_vector = self._embed(query)
        scores = torch.matmul(self.dtm, query_vector)
        topk_idx = scores.topk(k).indices.tolist()
        return topk_idx

In [19]:
gva = GloVeAverage(word2vec)
gva.train(squad_documents)
gva.add(squad_documents)
topk_idx = gva.search(squad_documents[0], 10)
print(topk_idx)

[0, 1, 8, 9, 6, 18, 24, 20, 7, 19]


> **Problem 3.2** *(2 points)* Use `GloVeAverage` to measure the recall at 10 for SQuAD validation dataset. How does it compare to TFIDF?

In [20]:
print(f"Recall@10 of GloVeAverage: {recall(gva, squad_queries, 10):.4f}")

Recall@10 of GloVeAverage: 0.6833


> It gives much lower score than TFIDF

> **Problem 3.3** *(2 points)*
Implement `GloVeAverageFaiss` that subclasses `SimilaritySearch` and uses Faiss `IndexFlatIP` instead of PyTorch native tensor operation for search.
Refer to the [Faiss wiki](https://github.com/facebookresearch/faiss/wiki/Getting-started) for instructions.

In [21]:
try:
    import faiss
except ImportError:
    !pip install faiss-cpu
    import faiss

In [22]:
class GloVeAverageFaiss(GloVeAverage):
    def train(self, documents: list):
        self.dtm = faiss.IndexFlatIP(dim)
        self.is_trained = True

    def add(self, documents: list):
        self._check_trained()
        sub_dtm = torch.stack([self._embed(document) for document in documents])
        self.dtm.add(sub_dtm.numpy())

    def search(self, query: str, k: int) -> list:
        query_vector = self._embed(query).unsqueeze(dim=0)
        scores = self.dtm.search(query_vector.numpy(), k)
        topk_idx = scores[1][0].tolist()
        return topk_idx

In [23]:
gvaf = GloVeAverageFaiss(word2vec)
gvaf.train(squad_documents)
gvaf.add(squad_documents)
topk_idx = gvaf.search(squad_documents[0], 10)
print(topk_idx)

[0, 1, 8, 9, 6, 18, 24, 20, 7, 19]


In [24]:
print(f"Recall@10 of GloVeAverageFaiss: {recall(gvaf, squad_queries, 10):.4f}")

Recall@10 of GloVeAverageFaiss: 0.6833


> **Problem 3.4** *(1 points)*
Compare the speed between `GloVeAverage` and `GloVeAverageFaiss` on SQuAD.
To make the measurement accurate, perform search many times (at least more than 1000) and take the average.

In [25]:
import timeit
from itertools import cycle

def benchmark(model, queries: list, k: int = 10, num_iter: int = 100000):
    iter_q = cycle(query for _, query in queries)
    total = timeit.timeit(lambda: model.search(next(iter_q), k), number=num_iter)
    return total / num_iter * 1000

print(f"GloVeAverage:      {benchmark(gva, squad_queries):.3f} ms / query")
print(f"GloVeAverageFaiss: {benchmark(gvaf, squad_queries):.3f} ms / query")

GloVeAverage:      0.237 ms / query
GloVeAverageFaiss: 1.089 ms / query
